<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/ANOVA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [2]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
#df_train_A = df_train[df_train['Histology'] == 'adenocarcinoma']
#df_test_A = df_test[df_test['Histology'] == 'adenocarcinoma']

#df_train_L = df_train[df_train['Histology'] == 'large cell']
#df_test_L = df_test[df_test['Histology'] == 'large cell']

#df_train_S = df_train[df_train['Histology'] == 'squamous cell carcinoma']
#df_test_S = df_test[df_test['Histology'] == 'squamous cell carcinoma']

In [0]:
#public_data_A = df_train_A.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#public_data_L = df_train_L.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#public_data_S = df_train_S.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
#PA_data_A = df_test_A.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#PA_data_L = df_test_L.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#PA_data_S = df_test_S.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_data = df_train.drop(['Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
PA_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_data_1 = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_labels = df_train.Histology

In [0]:
PA_labels = df_test.Histology

In [0]:
feat_A = public_data['Maximum'][public_data.Histology == 'adenocarcinoma']

In [0]:
feat_L = public_data['Maximum'][public_data.Histology == 'large cell']

In [0]:
feat_S = public_data['Maximum'][public_data.Histology == 'squamous cell carcinoma']

In [38]:
# load packages
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns F and P-value
fvalue, pvalue = stats.f_oneway(feat_A, feat_L, feat_S)
print(fvalue, pvalue)

4.7184035638853965 0.010540043241126989


In [42]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue, pvalue = stats.f_oneway(feat_A, feat_L, feat_S)

  if pvalue <= 0.05:
    print(fvalue, pvalue, column)


3.260231291649289 0.0415894832636264 HighGrayLevelEmphasis
4.7184035638853965 0.010540043241126989 Maximum
5.359338541197532 0.005818378887600915 Range
3.635718433558213 0.029123760736867767 LongRunHighGrayLevelEmphasis
3.187154836462757 0.04458597869558036 HighGrayLevelRunEmphasis
4.652836901674387 0.011204051595582876 GrayLevelVariance.2
3.1553089945578137 0.04595955606831947 SmallAreaHighGrayLevelEmphasis
4.845308726734106 0.009366122195029402 ZoneEntropy


Null hypotheses: Groups means are equal (no variation in means of groups)


Alternative hypotheses: At least, one group mean is different from other groups

Soglia che si considera in genere: 0.05

se pvalue <= 0.5 si rigetta l'Hp zero


##Compute the Kruskal-Wallis H-test for independent samples.

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA

In [44]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue, pvalue = stats.kruskal(feat_A, feat_L, feat_S)

  if pvalue <= 0.05:
    print(fvalue, pvalue, column)


6.147661147655405 0.04624367533780151 Maximum2DDiameterRow
7.327260213194336 0.02563927030450879 HighGrayLevelEmphasis
6.621721927698957 0.03648474817830056 Idn
6.620504649169675 0.036506960987710696 Autocorrelation
6.444153541861307 0.0398721668601435 Median
6.809322290739744 0.033218074273387084 Maximum
7.715868105731206 0.021111569887821632 90Percentile
8.575330349975955 0.013736961284395596 Range
6.975503981330917 0.03056951554313273 LongRunHighGrayLevelEmphasis
6.768094930660595 0.03390992734838747 HighGrayLevelRunEmphasis
11.13192988155447 0.0038258870091253303 GrayLevelVariance.2
7.892243014763665 0.019329526002898275 SmallAreaHighGrayLevelEmphasis
7.1864424655768175 0.0275095728076037 HighGrayLevelZoneEmphasis
10.247500028441095 0.005953654690921986 ZoneEntropy


In [47]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue_1, pvalue_1 = stats.f_oneway(feat_A, feat_L, feat_S)
  fvalue_2, pvalue_2 = stats.kruskal(feat_A, feat_L, feat_S)


  if (pvalue_1 <= 0.05) & (pvalue_2 <= 0.5):
    print(column)

HighGrayLevelEmphasis
Maximum
Range
LongRunHighGrayLevelEmphasis
HighGrayLevelRunEmphasis
GrayLevelVariance.2
SmallAreaHighGrayLevelEmphasis
ZoneEntropy
